<a href="https://colab.research.google.com/github/Santosh-Gupta/ScientificSummarizationDataSets/blob/master/BertExtractiveTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import requests
import os
import random
import tarfile
import pandas as pd
import zipfile
import multiprocess
import json
from glob import glob

In [0]:
!pip install torch
!pip install pytorch-pretrained-bert
!pip install tensorboardX
!pip install pyrouge

Download and Unzip Scibert weights

In [0]:
#Download SciBert weights and vocab file
import urllib.request
# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar', 'scibert.tar')

('scibert.tar', <http.client.HTTPMessage at 0x7fb455b18f60>)

In [0]:
import tarfile
tar = tarfile.open('scibert.tar', "r:")
tar.extractall()
tar.close()

In [0]:
os.listdir('scibert_scivocab_uncased')

['vocab.txt', 'weights.tar.gz']

In [0]:
tar = tarfile.open('scibert_scivocab_uncased/weights.tar.gz', "r:gz")
tar.extractall('scibert_scivocab_uncased')
tar.close()

In [0]:
os.path.abspath('../../scibert_scivocab_uncased/pytorch_model.bin')

'/content/scibert_scivocab_uncased/pytorch_model.bin'

In [0]:
os.path.abspath('scibert_scivocab_uncased/bert_config.json')

'/content/BertSum/src/scibert_scivocab_uncased/bert_config.json'

In [0]:
!git clone https://github.com/Santosh-Gupta/BertSum.git

Cloning into 'BertSum'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 423 (delta 50), reused 0 (delta 0), pack-reused 345
Receiving objects: 100% (423/423), 15.07 MiB | 17.55 MiB/s, done.
Resolving deltas: 100% (259/259), done.


In [0]:
%cd BertSum/src

/content/BertSum/src/BertSum/src


In [0]:
os.listdir( )

['models', 'train.py', 'others', 'distributed.py', 'prepro', 'preprocess.py']

In [0]:
os.listdir( '../../gdrive/My Drive/FinalBertSummarizationDataPart1')[0:10]

FileNotFoundError: ignored

In [0]:
#batch size 1024 with 50000 went over 12 hours, maybe 12:25

# 4096

In [0]:
os.path.abspath('../../gdrive/My Drive/BertSumCheckPointFiles')

In [0]:
#first time training no, 'train_from'. Updated for SciBert

# !python train.py -mode train -encoder transformer -dropout 0.2 -bert_data_path  '/content/gdrive/My Drive/FinalBertSummarizationDataPart1/Files' -model_path '/content/gdrive/My Drive/BertSumCheckPointFiles' -bert_config_path '/content/scibert_scivocab_uncased/bert_config.json' -pretrained_dir '/content/scibert_scivocab_uncased/weights.tar.gz' -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 500 -save_checkpoint_steps 50000 -batch_size 512 -decay_method noam -train_steps 50000 -accum_count 2 -log_file bert_transformer -use_interval true -warmup_steps 100 -ff_size 2048 -inter_layers 2 -heads 8

In [0]:
os.listdir()

In [0]:
#Figure out which GPU used
!nvidia-smi

In [0]:
#

In [0]:
#Training from checkpoint. Updated for the SciBert pretrained and config. 

#There are one of three training files directories 
# bertDataPath = '/content/gdrive/My Drive/FinalBertSummarizationDataPart1/Files'
bertDataPath = '/content/gdrive/My Drive/FinalBertSummarizationDataPart2/Files'
# bertDataPath = '/content/gdrive/My Drive/FinalBertSummarizationDataPart3/Files'

# Try os.environ['trainSave'] = 160000

# trainSave = 180000

modelPath = '/content/gdrive/My Drive/BertSumCheckPointFiles'

bertConfigPath = '/content/scibert_scivocab_uncased/bert_config.json'

pretrainedDir = '/content/scibert_scivocab_uncased/weights.tar.gz'

trainFrom = '/content/gdrive/My Drive/BertSumCheckPointFiles/model_step_280000.pt'

# batchSize = 8192

!python train.py -mode train -encoder transformer -dropout 0.2 -bert_data_path '$bertDataPath' \
-model_path '$modelPath' -bert_config_path '$bertConfigPath' -pretrained_dir '$pretrainedDir' \
-train_from '$trainFrom' -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 \
-report_every 1000 -save_checkpoint_steps 285000 -batch_size 4096 -decay_method noam -train_steps 285000 \
-accum_count 2 -log_file bert_transformer -use_interval true -warmup_steps 100 -ff_size 2048 -inter_layers 2 -heads 8

In [0]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [0]:
!nvidia-smi

In [0]:
# !python train.py -mode train -encoder transformer -dropout 0.2 -bert_data_path  '../../gdrive/My Drive/FinalBertSummarizationDataPart1/Files' -model_path '../../gdrive/My Drive/BertSumCheckPointFiles' -train_from 'model_step_5000.pt' -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 100 -save_checkpoint_steps 50 -batch_size 512 -decay_method noam -train_steps 50 -accum_count 2 -log_file bert_transformer -use_interval true -warmup_steps 100 -ff_size 2048 -inter_layers 2 -heads 8

In [0]:
!pip show pyrouge

In [0]:
!pyrouge_set_rouge_path '/usr/local/lib/python3.6/dist-packages/pyrouge/tests'

In [0]:
!git clone https://github.com/andersjo/pyrouge.git

In [0]:
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

In [0]:
#'/content/gdrive/My Drive/BertSumCheckPointFiles'

In [0]:
# bertDataPath = '/content/gdrive/My Drive/MiniTest/Files'

# trainSave = 150000

# modelPath = '/content/gdrive/My Drive/BertSumCheckPointFiles/'

# bertConfigPath = '/content/scibert_scivocab_uncased/bert_config.json'

# pretrainedDir = '/content/scibert_scivocab_uncased/weights.tar.gz'

# trainFrom = '/content/gdrive/My Drive/BertSumCheckPointFiles/model_step_130000.pt'

# logFile = '../logs/bert_transformer'

# testFrom = '/content/gdrive/My Drive/BertSumCheckPointFiles/model_step_130000.pt'

# resultsPath = '/content/gdrive/My Drive/BertSumResults/'

# !python train.py -mode test -bert_data_path '$bertDataPath' -model_path '$modelPath' \
# -bert_config_path '$bertConfigPath' -pretrained_dir '$pretrainedDir' -visible_gpus 0 -gpu_ranks 0 \
# -batch_size 2000 -log_file '$logFile' -result_path '$resultsPath' \
# -block_trigram true -test_from '$testFrom'

In [0]:
!apt-get install libmagic-dev
!pip install python-magic
import magic
mime = magic.Magic(mime=True)

In [0]:
mime.from_file('/content/gdrive/My Drive/BertSumResults_step60000.candidate')

In [0]:
with open('/content/gdrive/My Drive/BertSumResults_step130000.candidate') as myfile:
    for i in range(10):
        print( next(myfile))

In [0]:
with open('/content/gdrive/My Drive/BertSumResults_step130000.gold') as myfile:
    for i in range(10):
        print( next(myfile))

In [0]:
os.listdir('../temp/tmpvw1am0ff/model')

In [0]:
import torch

In [0]:
dataset = torch.load('../../gdrive/My Drive/MiniTest/Files.test.12.bert.pt')

In [0]:
print(dataset[13])